## 1. GPU Setup in Colab

In [ ]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')
    

Found GPU at: /device:GPU:0


In [ ]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


## 2. Load and analyze data

The dataset used is from kaggle: https://www.kaggle.com/paultimothymooney/medical-speech-transcription-and-intent

This data contains thousands of audio utterances and corresponding transcriptions for common medical symptoms like “knee pain” or “headache”. Only the transcriptions are used in this project.

We can see from below:
* column "phrase" contains transcriptions describing a person's certain medical symptoms
* column "promp" contains their corresponding intents (25 intents in total)


In [ ]:
!pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

     |████████████████████████████████| 890kB 5.5MB/s 
     |████████████████████████████████| 890kB 29.9MB/s 
     |████████████████████████████████| 3.0MB 45.7MB/s 
     |████████████████████████████████| 1.1MB 53.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=23a34b0995ad105282132bf7f0353fe246fabf74009fd656a6a611c27ed868bd
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
data = pd.read_csv('gdrive/My Drive/overview-of-recordings.csv')
data1 = data[['phrase','prompt']]
data1.sample(5)

,phrase,prompt
2951,I feel back pain when I carry heavy things,Back pain
2758,I feel great pain in the head,Head ache
1794,There is a tingling sensation in my neck.,Neck pain
376,I get breakouts on my chest with red patches t...,Acne
6093,I feel discomfort throughout the body in general,Body feels weak


In [ ]:
df=data1.copy()
df.isna().sum()

phrase    0
prompt    0
dtype: int64

In [ ]:
df['prompt'].value_counts()

Acne                  328
Shoulder pain         320
Joint pain            318
Infected wound        306
Knee pain             305
Cough                 293
Feeling dizzy         283
Muscle pain           282
Heart hurts           273
Ear ache              270
Hair falling out      264
Head ache             263
Feeling cold          263
Skin issue            262
Stomach ache          261
Back pain             259
Neck pain             251
Internal pain         248
Blurry vision         246
Body feels weak       241
Hard to breath        233
Emotional pain        231
Injury from sports    230
Foot ache             223
Open wound            208
Name: prompt, dtype: int64

In [ ]:
print('Total number of intents: %d'%(len(df['prompt'].value_counts().index)))

Total number of intents: 25


## 3. Split data to train, validation and test sets

I split data to train(70%), validation(10%) and testset (20%) stratified by the variable "intent". After stratification, data for each intent will balanced and data for each set will be proportional to 70%, 10% and 20%. That is crucial for training and testing purposes.

In [ ]:
from sklearn.model_selection import train_test_split

X, sentence_test, y, intent_test = train_test_split(df.phrase, df.prompt, stratify = df.prompt,test_size=0.2, random_state=4612)
sentence_train, sentence_val, intent_train, intent_val = train_test_split(X, y, stratify = y,test_size=0.125, random_state=4612)


In [ ]:
print(f"#examples in training set:{ sentence_train.shape[0]}\n#examples in validation set:{ sentence_val.shape[0]}\n#examples in test set:{ sentence_test.shape[0]}")

#examples in training set:4662
#examples in validation set:666
#examples in test set:1333


## 4. Tokenization and input formatting

I Prepare the input data to the correct format before training as follows:
* tokenizing all sentences 
* padding and truncating all sentences to the same length. 
* Creating the attention masks which explicitly differentiate real tokens from [PAD] tokens. 0 or 1.
* encoding the label "intent" to numbers. 25 intents to 25 numbers.
* creating DataLoaders for our training, validation and test sets

In [ ]:
TRAIN_BATCH_SIZE =32
VALID_BATCH_SIZE = 64
EPSILON = 1e-08
EPOCHS = 4
LEARNING_RATE = 2e-5
SEED = 1215
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
max_len = 0
input = []
length=[]
for sent in sentence_train:

    input_ids = tokenizer.encode(sent, add_special_tokens=True)
    input.append(input_ids)
    length.append(len(input_ids))
    max_len = max(max_len, len(input_ids))
    mean_len = sum(length)/len(length)
print('Max sentence length:%d \nMean sentence length:%d' % (max_len,mean_len))

Max sentence length:39 
Mean sentence length:14


In [ ]:
def tokenize(sentence):
  batch = tokenizer(list(sentence),             
                  is_pretokenized=False,
                  padding=True, 
                  truncation=True,
                  return_tensors="pt")
  return batch

In [ ]:
tok_train = tokenize(sentence_train)
tok_val = tokenize(sentence_val)
tok_test = tokenize(sentence_test)


In [ ]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
label_train = torch.tensor((LE.fit_transform(intent_train)))
label_val = torch.tensor((LE.fit_transform(intent_val)))
label_test = torch.tensor((LE.fit_transform(intent_test)))


In [ ]:
from torch.utils.data import TensorDataset

train_dataset = TensorDataset(tok_train['input_ids'], tok_train['attention_mask'],label_train)
validation_dataset = TensorDataset(tok_val['input_ids'], tok_val['attention_mask'],label_val)
test_dataset = TensorDataset(tok_test['input_ids'], tok_test['attention_mask'],label_test)


In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

train_dataloader = DataLoader(
            train_dataset,  
            sampler = RandomSampler(train_dataset), 
            batch_size = TRAIN_BATCH_SIZE 
        )

validation_dataloader = DataLoader(
            validation_dataset, # validation samples.
            sampler = SequentialSampler(validation_dataset), 
            batch_size = VALID_BATCH_SIZE # Evaluate with this batch size.
        )

test_dataloader = DataLoader(
            validation_dataset, 
            sampler = SequentialSampler(validation_dataset), 
            batch_size = VALID_BATCH_SIZE 
        )

## 5. Train BERT classification model

I use BertForSequenceClassification, a BERT model with an added single linear layer on top for classification. As we feed input data, the entire pre-trained BERT model and the additional untrained classification layer is trained on our specific task.



In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels = 25)
model.cuda() 

optimizer = AdamW(model.parameters(), 
                  lr = LEARNING_RATE, 
                  eps = EPSILON

from transformers import get_linear_schedule_with_warmup

total_steps = len(train_dataloader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:

def calcuate_accu(big_idx, targets):
    n_correct = (big_idx==targets).sum().item()
    return n_correct

In [ ]:
import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))   
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('runs/Tensorboard')

In [ ]:
import random
import torch

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
def train(epochs):
  total_t0 = time.time() 
  tr_loss = 0
  n_correct = 0
  nb_tr_steps = 0
  nb_tr_examples = 0
  
  for epoch in range(0, epochs):
      print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
      print('Training...')

      t0 = time.time()     
      total_tr_loss = 0
      total_n_correct = 0
      total_nb_tr_examples = 0
      model.train()    

      for step, batch in enumerate(train_dataloader, 0):     
          input_ids = batch[0].to(device, dtype = torch.long)
          input_mask = batch[1].to(device, dtype = torch.long)
          labels = batch[2].to(device, dtype = torch.long)

          model.zero_grad()       

          outputs = model(input_ids, token_type_ids=None, attention_mask=input_mask)
          loss_function = torch.nn.CrossEntropyLoss()
          loss = loss_function(outputs[0], labels) 
          tr_loss += loss.item() 
          total_tr_loss += loss.item()
          big_val, big_idx = torch.max(outputs[0], dim=1)
          n_correct += calcuate_accu(big_idx, labels)  
          total_n_correct += calcuate_accu(big_idx, labels)                  
          nb_tr_steps += 1
          nb_tr_examples+=labels.size(0)
          total_nb_tr_examples+=labels.size(0)

          if step % 20==19:
              loss_step = tr_loss/nb_tr_steps
              accu_step = n_correct/nb_tr_examples  
              print(f"Training Loss per 20 steps(batches): {loss_step}")
              print(f"Training Accuracy per 20 steps(batches): {accu_step}")
              elapsed = format_time(time.time() - t0)      
              print('Batch {} of {}.  Elapsed: {:}.'.format(step+1, len(train_dataloader), elapsed))
              tr_loss = 0;n_correct = 0;nb_tr_steps = 0;nb_tr_examples = 0
                
          loss.backward() 
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)  # prevent the "exploding gradients" problem.
          optimizer.step()
          scheduler.step() 

      train_loss_per_epoch = total_tr_loss / len(train_dataloader)            
      train_accuracy_per_epoch=total_n_correct/total_nb_tr_examples
      training_time = format_time(time.time() - t0)

      print("")
      print("training loss per epoch: {0:.2f}".format(train_loss_per_epoch))
      print("training accuracy per epoch: {0:.2f}".format(train_accuracy_per_epoch))
      print("Training 1 epcoh took: {:}".format(training_time))

In [ ]:
train(epochs = EPOCHS)

======== Epoch 1 / 4 ========
Training...
Training Loss per 20 steps(batches): 3.207607901096344
Training Accuracy per 20 steps(batches): 0.0671875
Batch 20 of 146.  Elapsed: 0:00:05.
Training Loss per 20 steps(batches): 3.1109325170516966
Training Accuracy per 20 steps(batches): 0.0859375
Batch 40 of 146.  Elapsed: 0:00:09.
Training Loss per 20 steps(batches): 2.941100037097931
Training Accuracy per 20 steps(batches): 0.1578125
Batch 60 of 146.  Elapsed: 0:00:14.
Training Loss per 20 steps(batches): 2.7166395783424377
Training Accuracy per 20 steps(batches): 0.2734375
Batch 80 of 146.  Elapsed: 0:00:18.
Training Loss per 20 steps(batches): 2.4430582165718078
Training Accuracy per 20 steps(batches): 0.4875
Batch 100 of 146.  Elapsed: 0:00:23.
Training Loss per 20 steps(batches): 2.191276413202286
Training Accuracy per 20 steps(batches): 0.6421875
Batch 120 of 146.  Elapsed: 0:00:27.
Training Loss per 20 steps(batches): 1.9297438085079193
Training Accuracy per 20 steps(batches): 0.75625

## 6. Test the model on the validation set

In [ ]:
def valid(model, validation_loader):
  model.eval()
  val_loss = 0
  nb_val_examples = 0
  n_correct = 0
  with torch.no_grad():
    for _, data in enumerate(validation_loader, 0): 
      ids = data[0].to(device, dtype = torch.long)
      mask = data[1].to(device, dtype = torch.long)
      targets = data[2].to(device, dtype = torch.long)
      outputs = model(ids, mask)
      loss_function = torch.nn.CrossEntropyLoss()
      loss = loss_function(outputs[0], targets)
      val_loss += loss.item()
      big_val, big_idx = torch.max(outputs[0], dim=1)
      n_correct += calcuate_accu(big_idx, targets)
      nb_val_examples+=targets.size(0)

  val_ave_loss = val_loss/len(validation_loader)
  val_accu = (n_correct*100)/nb_val_examples
  print("Loss on validation/test data: %0.2f" % val_ave_loss)
  print("Accuracy on validation/test data: %0.2f%%" % val_accu)
  
  return

In [ ]:
valid(model, validation_dataloader)

Loss on validation/test data: 0.14
Accuracy on validation/test data: 99.40%


## 7. Obtain test error

In [ ]:
valid(model, test_dataloader)

Loss on validation/test data: 0.14
Accuracy on validation/test data: 99.40%


## 8. Save the model, tokenizer and labels

In [ ]:
import os


output_dir = './Documents/intent_detection_healthcare_bert/saved_bert_model_and_tokenizer/'

if not os.path.exists(output_dir):
  os.makedirs(output_dir)

print("Saving model to %s" % output_dir)

model_to_save = model.module if hasattr(model, 'module') else model  # distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)



Saving model to ./Documents/intent_detection_healthcare_bert/saved_bert_model_and_tokenizer/


('./Documents/intent_detection_healthcare_bert/saved_bert_model_and_tokenizer/vocab.txt',
 './Documents/intent_detection_healthcare_bert/saved_bert_model_and_tokenizer/special_tokens_map.json',
 './Documents/intent_detection_healthcare_bert/saved_bert_model_and_tokenizer/added_tokens.json')

In [ ]:
df_label = pd.DataFrame(tuple(zip(range(25),LE.classes_)), columns=['id','intent'])
df_label.to_pickle('./Documents/intent_detection_healthcare_bert/saved_bert_model_and_tokenizer/df_label.pkl')

In [ ]:
!cp -r ./Documents/intent_detection_healthcare_bert/saved_bert_model_and_tokenizer/ "gdrive/My Drive/"

## 9. Prepare the model for deployment
* Load the saved model, tokenizer and labels 
* Create a medical_symptom_detector function with the loaded model, tokenizer and labels, which helps predict the medical intent of a medical message. 
* test the detector on an unseen example

In [ ]:
!pip install transformers
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification

input_dir = 'gdrive/My Drive/saved_bert_model_and_tokenizer/'

loaded_model = BertForSequenceClassification.from_pretrained(input_dir)
loaded_model.eval()
loaded_tokenizer = BertTokenizer.from_pretrained(input_dir)
loaded_df_label = pd.read_pickle('gdrive/My Drive/saved_bert_model_and_tokenizer/df_label.pkl')



In [ ]:

def medical_symptom_detector(intent):

  pt_batch = loaded_tokenizer(
  intent,
  padding=True,
  truncation=True,
  return_tensors="pt")

  pt_outputs = loaded_model(**pt_batch)
  __, id = torch.max(pt_outputs[0], dim=1)
  prediction = loaded_df_label.iloc[[id.item()]]['intent'].item()
  print('You may have a medical condition: %s. Would you like me to transfer your call to your doctor?'%(prediction))
  return 

In [ ]:
input = 'my left knee hurts so much'
medical_symptom_detector(input)

You may have a medical condition: Knee pain. Would you like me to transfer your call to your doctor?
